In [ ]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

In [ ]:
#for Fernanda's part
%%bash

pip install PyPDF2
pip install nltk
pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 61.1 MB/s eta 0:00:00


In [ ]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [ ]:
#initializing the document store:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
#Fernanda's code for loading the pdf and preprocessing

from google.colab import files

import PyPDF2
import fitz
import nltk
from collections import Counter
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

# Using a technique called n-gram extraction

from nltk import bigrams
from nltk import FreqDist

uploaded = files.upload()

def extract_text(file_path):
    # Using PyMuPDF to extract text
    text = ""
    with fitz.open(file_path) as pdf_file:
        for page_num in range(len(pdf_file)):
            page = pdf_file.load_page(page_num)
            text += page.get_text()
    return text

def extract_metadata(file_path):
    # Using PyMuPDF for extracting metadata
    with fitz.open(file_path) as pdf_file:
        metadata = pdf_file.metadata
        number_of_pages = len(pdf_file)
        return metadata, number_of_pages

def preprocess_text(text):
    # Lowercase
    text = text.lower()
    # Punctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    # Remove common words
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])
    return text

def identify_keywords_and_expressions(text):
    words = text.split()
    # Identifying frequent words
    frequent_words = Counter(words).most_common(50)

    excluded_words = {"was", "will", "they", "are", "used", "number", "many", "member", "shown", "done", "blue", "eyes", "using", "use"}
    keywords = [word for word, count in frequent_words if word not in excluded_words and len(word) > 2 and len(word) < 15]

    # Extracting bigrams (expressions)
    bigram_list = list(bigrams(words))
    bigram_freq = FreqDist(bigram_list).most_common(50)
    expressions = [' '.join(bigram) for bigram, count in bigram_freq]

    # List of the keywords and expressions
    print("List of keywords:", keywords)
    print("List of expressions:", expressions)
    return keywords, expressions



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Saving test7.pdf to test7.pdf


In [ ]:
# Arpad, I changed this part as well, see if it makes sense
#part to save the the extracted/preprocessed text to an output.txt file which goes to the document store

file_path = next(iter(uploaded))

text = extract_text(file_path)
metadata, number_of_pages = extract_metadata(file_path)

text = preprocess_text(text)

keywords, expressions = identify_keywords_and_expressions(text)

# Checking if metadata function works
print("Number of Pages:", number_of_pages)
print("Length of Text:", len(text))
print("Title:", metadata.get("title", "Not available"))
print("Author:", metadata.get("author", "Not available"))
print("Summary:", metadata.get("subject", "Not available"))

List of keywords: ['data', 'table', 'graphs', 'charts', 'tables', 'summary', 'audience', 'presentation', 'understand', 'graph', 'chart', 'presenting', 'patterns', 'information', 'statistics', 'objective', 'components', 'types', 'present', 'trends', 'create', 'analysis', 'layout', 'form', 'students', 'categorical', 'grade', 'percentage', 'show', 'interpret', 'clear', 'statistical', 'different', 'step', 'forms', 'dimensions', 'purpose', 'type', 'consider', 'indicators', 'communicate', 'manner', 'easy', 'useful']
List of expressions: ['graphs charts', 'charts graphs', 'table •', 'charts •', 'summary table', 'graph chart', 'audience •', 'presenting data', 'data •', 'patterns trends', 'multidimensional tables', 'types graphs', 'objective audience', 'forms presentation', '• tables', '• components', 'components table', 'present data', 'data using', 'form •', '• summary', 'summary tables', 'categorical variable', 'students grade', '– simple', 'multidimensional summary', 'tables •', '• show', '

In [ ]:
#write the text to output.txt
with open("output.txt", "w", encoding='utf-8') as text_file:
  text_file.write(text)

print("Text extracted, preprocessed and saved to output.txt!")

Text extracted, preprocessed and saved to output.txt!


In [ ]:
#writing the document(s) to the documentstore
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

# Specify the path to the single text file you want to index
path_to_file = "output.txt"  # Replace with the actual file path
indexing_pipeline = TextIndexingPipeline(document_store)
# Directly pass the file path to run_batch, as it accepts both lists and individual paths
indexing_pipeline.run_batch(file_paths=[path_to_file])  # Enclose in a list for clarity

INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Updating BM25 representation...: 100%|██████████| 6/6 [00:00<00:00, 1853.56 docs/s]


{'documents': [<Document: {'content': 'introduction data analysis greg keeble unesco institute statistics overview • objective audience • forms presentation • tables dimensions • components table • layout table • presenting data • purpose graphs charts • components graphs charts • types graphscharts • data presentation objective audience • type statistics depends main objective presentation target audience • consider best present data indicators – trying communicate – audience – kind presentation effective – help audience better understand data forms presentation communicate audience meaning data using summary statistics informative interesting manner easy understand • tables useful presenting data statistics numeric form • charts graphs may used highlight key patterns trends graphical form • descriptive text describe summarize findings verbal form dimensions summary table • summary tables used present counts students teachers schools categorical variable • eg number teachers qualifica

In [ ]:
#initializing the retriever
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [ ]:
from haystack.nodes import FARMReader
#if use_gpu=True GPU accerelation (in Edit: Notebook settings) must be enabled
reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1
INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'deepset/roberta-base-squad2' (Roberta)
INFO:haystack.modeling.model.language_model:Auto-detected model language: english
INFO:haystack.modeling.model.language_model:Loaded 'deepset/roberta-base-squad2' (Roberta model) from model hub.
INFO:haystack.modeling.utils:Using devices: CUDA:0 - Number of GPUs: 1


In [ ]:
#creating the retriever-reader pipeline
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [ ]:
#asking a question:
prediction = pipe.run(query="What is this report about?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}})

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  1.34 Batches/s]


In [ ]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'international journal innovative technology', 'type': 'extractive', 'score': 0.33749920129776, 'context': 'howing box plot manufacturer reviewsrating data frame international journal innovative technology exploring engineering ijitee issn 22783075 volume8 i', 'offsets_in_document': [{'start': 3886, 'end': 3929}], 'offsets_in_context': [{'start': 54, 'end': 97}], 'document_ids': ['e7ccf0338259516ca95d3b8db3be9f05'], 'meta': {'_split_id': 0}}>,
             <Answer {'answer': 'innovative technology exploring engineering ijitee issn 22783075 volume8 issue12 october 2019 4727 published blue eyes intelligence engineering sciences', 'type': 'extractive', 'score': 0.2875562310218811, 'context': 'innovative technology exploring engineering ijitee issn 22783075 volume8 issue12 october 2019 4727 published blue eyes intelligence engineering sciences', 'offsets_in_document': [{'start': 22, 'end': 174}], 'offsets_in_context': [{'start': 0, 'end': 152}], 'document_id

In [ ]:
from haystack.utils import print_answers

print_answers(prediction, details="minimum")  ## Choose from `minimum`, `medium`, and `all`

'Query: What is this report about?'
'Answers:'
[   {   'answer': 'international journal innovative technology',
        'context': 'howing box plot manufacturer reviewsrating data frame '
                   'international journal innovative technology exploring '
                   'engineering ijitee issn 22783075 volume8 i'},
    {   'answer': 'innovative technology exploring engineering ijitee issn '
                  '22783075 volume8 issue12 october 2019 4727 published blue '
                  'eyes intelligence engineering sciences',
        'context': 'innovative technology exploring engineering ijitee issn '
                   '22783075 volume8 issue12 october 2019 4727 published blue '
                   'eyes intelligence engineering sciences'},
    {   'answer': 'sales would increase temperature rose',
        'context': ' study parameters mean parameter slope indicating amount '
                   'sales would increase temperature rose degree number seems '
                